<a href="https://colab.research.google.com/github/ragabhumi/Pelatihan_Earth_Imaging/blob/main/Pemodelan_Magnet_Bumi_Data_Model_Global.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PELATIHAN TEKNIS EARTH IMAGING - 2023**

**Pemodelan Magnet Bumi menggunakan Data Model Global EMAG2v3**

oleh: Yosi Setiawan

Bogor, 16 Juni 2023

Pelatihan ini bertujuan untuk memberikan pemahaman dan keahlian dalam melakukan analisis & interpretasi kondisi bawah permukaan bumi menggunakan metode magnet bumi dan aplikasi yang mendukung Earth Imaging guna menunjang tugas dan fungsi sebagai Personal Geofisika.

Pada praktik pengolahan data magnet bumi ini, software yang digunakan adalah Fatiando a Terra (Uieda et.al., 2013) untuk analisis data magnet bumi, dan PyGMT (Uieda et.al., 2023) untuk visualisasi yang keduanya berbasis Python. Pengolahan dilakukan di Google Colab untuk memudahkan akses terhadap program dan data yang digunakan tanpa harus menginstall di komputer atau laptop masing-masing peserta.

Pada bagian ini, data yang akan digunakan adalah **data model magnet bumi EMAG2v3**.

Langkah pertama yang harus dilakukan adalah menginstal CondaColab di Google Colab di akun Google masing-masing. Tekan tombol **Run cell** di sebelah kiri setiap cell atau tekan tombol Ctrl+Enter. Tunggu hingga proses instalasi selesai. Jika muncul notifikasi **Your session crashed for an unknown reason.**, abaikan dan tutup notifikasi tersebut.

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Selanjutnya, cek apakah instalasi CondaColab telah berhasil. Pastikan hasil yang muncul adalah **Everything looks OK!**.

In [ ]:
import condacolab
condacolab.check()

Langkah berikutnya adalah menginstal modul-modul Python yang diperlukan. Proses ini memerlukan waktu yang agak lama. Tunggulah hingga proses selesai.

In [ ]:
!mamba install verde harmonica boule pooch pygmt pyproj pandas xarray xrft ensaio pyproj rioxarray

Import modul-modul Python yang akan digunakan

In [ ]:
# The standard Python science stack
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import xrft
import datetime
from scipy import signal

# For projections (wrapped for Proj)
import pyproj

# Plotting maps using GMT
import pygmt

# The Fatiando stack
import pooch
import verde as vd
import boule as bl
import harmonica as hm

Data magnet bumi yang akan digunakan dalam pelatihan ini telah diupload di Github. Untuk mengunduhnya kita gunakan modul Pooch. Terdapat 8 file yang tersedia, pilihlah sesuai dengan kelompok anda.

In [ ]:
url_mag = 'https://raw.githubusercontent.com/ragabhumi/Pelatihan_Earth_Imaging/main/data/EMAG2v3_area8.csv'
path_mag = pooch.retrieve(url_mag, known_hash=None)
print(path_mag)

Selanjutnya, data model magnet bumi dengan format csv kita parsing menggunakan modul Pandas dan diberikan nama untuk setiap kolomnya.

In [ ]:
data = pd.read_csv(path_mag, sep=' ', names=['i', 'j', 'Longitude', 'Latitude', 'Sealevel', 'UpCont', 'Code', 'Error'])
data

Pada data yang kita gunakan, terdapat beberapa titik yang datanya ambigu maupun titik yang tidak ada data. Titik yang datanya ambigu diberikan kode 888 dan nilainya -888 nT, sedangkan titik yang tidak ada data diberikan kode 999 dan nilainya 99999 nT. Sehingga titik-titik tersebut harus kita hilangkan.

In [ ]:
data = data[(data['Code'] != 888) & (data['Code'] != 999)]
data

Kita tampilkan titik-titik grid model magnet.

In [ ]:
# Kita atur batas koordinat sesuai dengan batasan data kita
lon_min = data['Longitude'].min()
lon_max = data['Longitude'].max()
lat_min = data['Latitude'].min()
lat_max = data['Latitude'].max()

fig = pygmt.Figure()
fig.basemap(projection='M20', region=[lon_min, lon_max, lat_min, lat_max], frame=True)
fig.coast(resolution='f', shorelines='2p,black,solid')
fig.plot(x=data['Longitude'], y=data['Latitude'],
    style="c0.1c",
    fill="red",
    pen="faint")
fig.show()

Menampilkan peta model anomali magnet bumi dalam bentuk kontur. Pertama, kita interpolasikan dahulu data txt menjadi data grid.

In [ ]:
pre_data = pygmt.blockmean(x=data['Longitude'], y=data['Latitude'], z=data['UpCont'], region=[lon_min, lon_max, lat_min, lat_max], spacing='1m')
data_grid = pygmt.surface(pre_data, region=[lon_min, lon_max, lat_min, lat_max], spacing='1m')
data_grid

Kita tampilkan peta konturnya menggunakan PyGMT.

In [ ]:
fig = pygmt.Figure()
pygmt.makecpt(cmap="roma+h", reverse=True, series=[data_grid.min().values, data_grid.max().values], background=True)
fig.basemap(projection='M20', region=[lon_min, lon_max, lat_min, lat_max], frame=True)
fig.grdimage(grid=data_grid, cmap=True)
fig.coast(resolution='f', shorelines='2p,black,solid')
fig.grdcontour(grid=data_grid, interval=50, annotation=50)

# Add colorbar
fig.colorbar(frame='af+l"Peta Anomali Medan Magnet (nT)"', position="JBC+h+o0/1c+e")
fig.show()

**1. Reduksi ke Kutub**

Data anomali magnet bumi ini harus kita reduksi ke kutub sebelum dilakukan pemisahan anomali regional dan residual. Namun, proses reduksi ke kutub ini hanya dapat dilakukan pada sistem koordinat kartesian, sehingga data dalam sistem koordinat geografis harus kita proyeksikan ke sistem koordinat kartesian terlebih dahulu.

Kita definisikan dahulu proyeksinya mengunakan pyproj.

In [ ]:
CRS = pyproj.crs.CRS(proj="merc", lat_ts=data_grid.lat.mean().values)

Selanjutnya, kita konversi data dari sistem koordinat geografis ke sistem koordinat kartesian.

In [ ]:
data_grid.rio.write_crs('+proj=lonlat', inplace=True)
utm_grid = data_grid.rio.reproject(CRS)
utm_grid = utm_grid.rename({'x':'easting', 'y':'northing'})
utm_grid

Kita lakukan transformasi reduksi ke kutub.

In [ ]:
# Pad the grid to increase accuracy of the FFT filter
pad_width = {
    "easting": utm_grid.easting.size // 3,
    "northing": utm_grid.northing.size // 3,
}

utm_grid_pad = xrft.pad(utm_grid, pad_width)

inclination, declination = -30.92, 0.52

rtp_grid = hm.reduction_to_pole(
    utm_grid_pad, inclination=inclination, declination=declination
)

# Unpad the reduced to the pole grid
rtp_grid = xrft.unpad(rtp_grid, pad_width)
rtp_grid

Data yang sudah ditransformasikan ke kutub dalam sistem koordinat kartesian kita proyeksikan kembali kedalam sistem koordinat geografis.

In [ ]:
rtp_grid.rio.write_crs(CRS, inplace=True)
rtp_grid.rio.set_spatial_dims("easting", "northing", inplace=True)
rtp_grid = rtp_grid.rio.reproject('+proj=latlon')
rtp_grid

Menampilkan peta anomali reduksi ke kutub.

In [ ]:
fig = pygmt.Figure()
pygmt.makecpt(cmap="roma+h", reverse=True, series=[rtp_grid.min().values, rtp_grid.max().values], background=True)
fig.basemap(projection='M20', region=[lon_min, lon_max, lat_min, lat_max], frame=True)
fig.grdimage(grid=rtp_grid, cmap=True)
fig.coast(resolution='f', shorelines='2p,black,solid')
fig.grdcontour(grid=rtp_grid, interval=100, annotation=100)

# Add colorbar
fig.colorbar(frame='af+l"Peta Anomali Reduksi ke Kutub (nT)"', position="JBC+h+o0/1c+e")
fig.show()

**2. Penentuan Anomali Regional**

Untuk mendapatkan anomali residual, kita buat dahulu model anomali regionalnya. Disini kita akan menggunakan polinomial orde 2.

In [ ]:
xv, yv = np.meshgrid(rtp_grid.x, rtp_grid.y, indexing='ij')
coordinates = (xv, yv)
trend = vd.Trend(degree=2).fit(coordinates, rtp_grid.values)

regional = rtp_grid.copy()
regional = regional.rename('regional')
regional.values = trend.predict(coordinates)
regional

In [ ]:
fig = pygmt.Figure()
pygmt.makecpt(cmap="roma+h", reverse=True, series=[regional.min().values, regional.max().values], background=True)
fig.basemap(projection='M20', region=[lon_min, lon_max, lat_min, lat_max], frame=True)
fig.grdimage(grid=regional, cmap=True)
fig.coast(resolution='f', shorelines='2p,black,solid')
fig.grdcontour(grid=regional, interval=20, annotation=20)

# Add colorbar
fig.colorbar(frame='af+l"Peta Anomali Medan Magnet Regional (nT)"', position="JBC+h+o0/1c+e")
fig.show()

**3. Pemisahan anomali regional - residual**



In [ ]:
residual = rtp_grid - regional
residual

In [ ]:
fig = pygmt.Figure()
pygmt.makecpt(cmap="roma+h", reverse=True, series=[residual.min().values, residual.max().values], background=True)
fig.basemap(projection='M20', region=[lon_min, lon_max, lat_min, lat_max], frame=True)
fig.grdimage(grid=residual, cmap=True)
fig.coast(resolution='f', shorelines='2p,black,solid')
fig.grdcontour(grid=residual, interval=100, annotation=100)

# Add colorbar
fig.colorbar(frame='af+l"Peta Anomali Medan Magnet Residual (nT)"', position="JBC+h+o0/1c+e")
fig.show()

**Referensi**



> Arifin, Y. I. (2019). Kemagnetan Batuan (Anisotropi Batuan Beku). UNG Press Gorontalo.

> Constable, C.G., and Constable, S.C., (2004). Satellite magnetic field measurements: applications in studying the deep earth. In Sparks, R.S.J., and Hawkesworth, C.J., (eds.), The State of the Planet: Frontiers and Challenges in Geophysics. Washington, DC: American Geophysical Union, doi: 10.1029/150GM13, pp. 147–160.

> Fenoglio, M. A., Johnston, M.J.S., dan Byerlee, J.D. (1995). Magnetic and Electric Fields Associated with Changes in High Pore Pressure in Fault Zone-Application to the Loma Prieta ULF Emissions, J. Geophys. Res. 100, pp 12951-12958.

> Freund, F.T. (2007). Pre-Earthquake Signals-Part I: Deviatoric Stresses Turn Rocks into A Source of Electric Currents, Natural Hazards Earth System Science, pp 535-541.

> Hattori, K. (2004). ULF geomagnetic changes associated with large earthquakes, Terr. Atmos. OceanSci., vol. 15,no. 3, pp. 329–360.

> Ismaguilov, V. S., Kopytenko, Yu. A., Hattori, K., and Hayakawa, M. (2003). Variations of phase velocity and gradient values of ULF geomagnetic disturbances connected with the Izu strong earthquakes, Nat. Hazards Earth Syst. Sci., 3, 211–215, https://doi.org/10.5194/nhess-3-211-2003.

> Kolhatkar, A., Jamison, A., Litvinov, D., Willson, R., & Lee, T. (2013). Tuning the Magnetic Properties of Nanoparticles. International Journal of Molecular Sciences, 14(8), 15977–16009. MDPI AG. Retrieved from http://dx.doi.org/10.3390/ijms14081597

> Kopytenko, Y. & Ismagilov, V. & Hayakawa, Masashi & Smirnova, N. & Troyan, V. & Peterson, T.. (2001). Investigation of the ULF electromagnetic phenomena related to earthquakes: Contemporary achievements and the perspectives. Annali di Geofisica. 44. 325-334.

> Merzer, M., dan Klemperer, S.L. (1997). Modelling Low-Frequency Magnetik Field Prekursors to the Loma Prieta Earthquake with A Prekursory Increase in Fault Zone Conductivity, Pure Applied Geophysics, Vol. 150, pp 217-248.

> Molchanov, O.A., dan Hayakawa, M. (1998). On the Generation Mechanism of ULF Seismogenic Emissions, Physics Earth Planet International, Vol. 105, pp 210-210.

> Nettleton, L. L. (1962). Gravity and Magnetics for Geologists and Seismologists. AAPG Bulletin, Vol. 46, No. 10, pp. 1815-1838

> Ravat, D. (2007). Reduction to Pole. In: Gubbins, D., Herrero-Bervera, E. (eds) Encyclopedia of Geomagnetism and Paleomagnetism. Springer, Dordrecht. https://doi.org/10.1007/978-1-4020-4423-6_275

> Santosa, Bagus J., Mashuri, Sutrisno, Wahyu T., Wafi, A., Salim, R., dan Armi, R. (2012). Interpretasi Metode Magnetik untuk Penentuan Struktur Bawah Permukaan di Sekitar Gunung Kelud Kabupaten Kediri. Jurnal Penelitian Fisika dan Aplikasinya (JPFA), Vol. 2 No. 1, Juni 2012. ISSN: 2087-9946.

> Uieda, L., V. C. Oliveira Jr, and V. C. F. Barbosa (2013), Modeling the Earth with Fatiando a Terra, Proceedings of the 12th Python in Science Conference, pp. 91-98. doi:10.25080/Majora-8b375195-010

> Uieda, L., Tian, D., Leong, W. J., Schlitzer, W., Grund, M., Jones, M., Fröhlich, Y., Toney, L., Yao, J., Magen, Y., Tong, JH., Materna, K., Belem, A., Newton, T., Anant, A., Ziebarth, M., Quinn, J., and Wessel, P. (2023). PyGMT: A Python interface for the Generic Mapping Tools (Version 0.9.0). doi:10.5281/zenodo.7772533